In [1]:
import init
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common import simple_sbml
from SBMLLint.common.reaction import Reaction
from SBMLLint.tools import sbmllint
from SBMLLint.tools import print_reactions

import os
import numpy as np
import pandas as pd
from pulp import *
import matplotlib.pyplot as plt
import time

from scipy.linalg import lu, inv
from scipy.optimize import linprog

In [2]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.simple_sbml import SimpleSBML
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix

from SBMLLint.games.som import SOM
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction

Current Directory: /Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebooks


In [3]:
os.getcwd()

'/Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebooks'

In [4]:
# Load relevant bigg models
# bigg constants
BIGG_RESULTS = "results"
BIGG_ID = "bigg_id"
BIGG_GENECOUNT = "gene_count"
BIGG_REACTION_COUNT = "reaction_count"
BIGG_ORGANISM = "organism"
BIGG_METABOLITE_COUNT = "metabolite_count"
import requests
res = requests.get('http://bigg.ucsd.edu/api/v2/models')
bigg_models = res.json()[BIGG_RESULTS]
bigg_ids = [bigg[BIGG_ID] for bigg in bigg_models]
print("number of BiGG models: %d" % len(bigg_models))
bigg_models[:1]

number of BiGG models: 108


[{'bigg_id': 'e_coli_core',
  'gene_count': 137,
  'reaction_count': 95,
  'organism': 'Escherichia coli str. K-12 substr. MG1655',
  'metabolite_count': 72}]

In [5]:
bigg_df = pd.DataFrame(bigg_models).set_index(BIGG_ID)
bigg_df.sort_values(by=BIGG_REACTION_COUNT, 
                    inplace=True)
bigg_df.tail(20)

,gene_count,reaction_count,organism,metabolite_count
bigg_id,,,,
iECO111_1330,1328,2760,Escherichia coli O111:H- str. 11128,1959
iEcE24377_1341,1341,2763,Escherichia coli O139:H28 str. E24377A,1972
iEC1364_W,1364,2764,Escherichia coli W,1927
iECIAI1_1343,1343,2765,Escherichia coli IAI1,1968
iECSE_1348,1348,2768,Escherichia coli SE11,1957
iEcolC_1368,1368,2768,Escherichia coli ATCC 8739,1969
iUMNK88_1353,1353,2777,Escherichia coli UMNK88,1969
iEKO11_1354,1354,2778,Escherichia coli KO11FL,1972
iEC1368_DH5a,1368,2779,Escherichia coli DH5[alpha],1951


In [6]:
bigg_df.tail(30)

,gene_count,reaction_count,organism,metabolite_count
bigg_id,,,,
iECB_1328,1329,2748,Escherichia coli B str. REL606,1951
iEcDH1_1363,1363,2750,Escherichia coli DH1,1949
iEcHS_1320,1321,2753,Escherichia coli HS,1963
iECDH1ME8569_1439,1439,2755,Escherichia coli DH1,1950
iEC55989_1330,1330,2756,Escherichia coli 55989,1953
iEC1349_Crooks,1349,2756,Escherichia coli ATCC 8739,1946
iETEC_1333,1333,2756,Escherichia coli ETEC H10407,1962
iEC1372_W3110,1372,2758,Escherichia coli str. K-12 substr. W3110,1918
iECO103_1326,1327,2758,Escherichia coli O103:H2 str. 12009,1958


In [7]:
# define four bigg models depending on size
MODEL_1000 = "iCN718.xml"
MODEL_4000 = "iLB1027_lipid.xml"
MODEL_6000 = "iCHOv1.xml"
MODEL_10000 = "Recon3D.xml"
FOUR_BIGG = [MODEL_1000, MODEL_4000, MODEL_6000, MODEL_10000]
print("------------------------------------------------------")
for bigg in FOUR_BIGG:
  print(bigg_df.loc[bigg[:-4]])
  print("------------------------------------------------------")

------------------------------------------------------
gene_count                                  709
reaction_count                             1015
organism            Acinetobacter baumannii AYE
metabolite_count                            888
Name: iCN718, dtype: object
------------------------------------------------------
gene_count                                           1027
reaction_count                                       4456
organism            Phaeodactylum tricornutum CCAP 1055/1
metabolite_count                                     2172
Name: iLB1027_lipid, dtype: object
------------------------------------------------------
gene_count                        1766
reaction_count                    6663
organism            Cricetulus griseus
metabolite_count                  4456
Name: iCHOv1, dtype: object
------------------------------------------------------
gene_count                  2248
reaction_count             10600
organism            Homo sapiens
metabolite

In [8]:
# use Recon3D
# SAMPLE_MODEL = FOUR_BIGG[3]
# fpath = os.path.join(cn.BIGG_DIR, SAMPLE_MODEL)
# simple = SimpleSBML()
# simple.initialize(fpath)

In [9]:
def solveMILP(mat):
  prob = LpProblem("Finding_Unconserved_Metabolites", LpMaximize)
  species = list(mat.index)
  species_inclusion = pulp.LpVariable.dicts("species", species, cat="Binary")
  species_mass = pulp.LpVariable.dicts("mass", species, cat="Continuous")
  # objective function (to maximize the number of species)
  prob += lpSum([species_inclusion[i] for i in species])
  # constraint 1 (for each reaction, the sum(stoichiometry[i]*mass[i])=0)
  for reaction in mat.columns:
    prob += lpSum(sum([mat[reaction][species]*species_mass[species] for species in species_inclusion])) == 0
  # constraint 2 (species_inclusion is less than or equal to mass of each species)
  for species in species_mass.keys():
    prob += species_inclusion[species] <= species_mass[species]
  prob.solve()
  return prob
def getUnconservedMetabolites(milp_result):
  unconserved_metabolites = []
  for v in milp_result.variables():
    if v.varValue==0:
      if v.name[:7]=="species":
        unconserved_metabolites.append(v.name[8:])
  return unconserved_metabolites

In [10]:
# milp_start = time.time()
# s = StoichiometryMatrix(simple)
# milp_result = solveMILP(s.stoichiometry_matrix)
# unconserved_metabolites = getUnconservedMetabolites(milp_result)
# milp_end = time.time()
# milp_time = milp_end - milp_start
# print("MILP time: %f" % milp_time)

In [11]:
def analyzeBIGG(model_file):
  simple = SimpleSBML()
  simple.initialize(os.path.join(cn.BIGG_DIR, model_file))
  print("Model: %s" % model_file)
  xgames_start = time.time()
  m = GAMES_PP(simple)
  res = m.analyze(simple_games=False, error_details=False, suppress_message=True)
  xgames_end = time.time()
  xgames_time = xgames_end - xgames_start
  print("xGAMES time: %f" % xgames_time)
  if res:
    print("xGAMES found an error!")
  else:
    print("xGAMES didn't find an error!")
  lp_start = time.time()
  s = StoichiometryMatrix(simple)
  lp_res = s.isConsistent()
  lp_end = time.time()
  lp_time = lp_end - lp_start
  print("LP time: %f" % lp_time)
  if s.result.status!=0:
    print("LP found an error!")
  milp_start = time.time()
  milp_result = solveMILP(s.stoichiometry_matrix)
  unconserved_metabolites = getUnconservedMetabolites(milp_result)
  milp_end = time.time()
  milp_time = milp_end - milp_start
  print("MILP time: %f" % milp_time)
  return m, s, milp_result, unconserved_metabolites

In [12]:
def analyzeByGAMES(model_file):
  simple = SimpleSBML()
  simple.initialize(os.path.join(cn.BIGG_DIR, model_file))
  print("Model: %s" % model_file)
  xgames_start = time.time()
  m = GAMES_PP(simple)
  res = m.analyze(simple_games=False, error_details=False, suppress_message=True)
  xgames_end = time.time()
  xgames_time = xgames_end - xgames_start
  print("xGAMES time: %f" % xgames_time)
  if res:
    print("xGAMES found an error!")
  else:
    print("xGAMES didn't find an error!")
  lp_start = time.time()
  return m

In [47]:
games_1000 = analyzeByGAMES(MODEL_1000)

Model: iCN718.xml
xGAMES time: 92.743054
xGAMES found an error!


In [48]:
print(games_1000.error_summary)

[ErrorSummary(type='type3', errors=[R_OMPHHX: {M_2omph_c} -> {M_2ombzl_c}, R_HVE: {M_3hivcoa_c} -> {M_3mb2coa_c}, R_FCLPA: {M_fc1p_c} -> {M_lald__L_c}, R_ECOAH12: {M_2mp2coa_c} -> {M_3hibutcoa_c}]), ErrorSummary(type='echelon', errors=[R_GMNA: {M_gln__D_c} -> , R_ADCS:  -> {M_4adcho_c}, R_34HPPOR:  -> {M_hgentis_c}, R_CTLSE:  -> 2.00 {M_h2o2_c}, R_PROt4: {M_pro__L_e} -> , R_PDX5POi: {M_pdx5p_c} -> , R_PYAM5PO: {M_pyam5p_c} -> , R_AMID2:  -> {M_pac_c}, R_HGENT:  -> 2.00 {M_co2_c}, R_PGLYCP: {M_2pglyc_c} -> , R_HBCHLR:  -> {M_b2coa_c}, R_PYDXNO: {M_pydxn_c} -> , R_GLUt4: {M_glu__L_e} -> , R_PYDXPP:  -> {M_pydx_c}, R_PYDXO:  -> {M_pydam_c}, R_FBA: {M_f1p_c} -> , R_TRE6PH: {M_tre_e} -> , R_BAMPPALDOX: 0.44 {M_pydx5p_c} -> , R_PHETA1: 0.57 {M_genald_c} -> , R_ASP1DC:  -> 1.00 {M_bamppald_c}, R_DB4PS:  -> {M_db4p_c}, R_GLYALDDr: {M_glyald_c} -> , R_ALDD2x: {M_2dr1p_c=M_2dr5p_c} -> , R_FBA2: 1.53 {M_dhap_c=M_g3p_c} -> , R_PPC: 1.24 {M_2pg_c=M_3pg_c} -> , R_GLYCK:  -> 0.78 {M_cit_c=M_icit_c}, 

In [51]:
gr_1000 = GAMESReport(games_1000)

In [53]:
gr_1000.reportEchelonError([games_1000.echelon_errors[0]])[1]

[304]

In [61]:
limit_repeat = 0
found_error = False
while (found_error is False) and (limit_repeat<20):
  limit_repeat += 1

#   games_1000 = analyzeByGAMES(MODEL_1000)
# for er in games_1000.echelon_errors:
#   x, y = gr_1000.reportEchelonError([er])
#   if y[0]<10:
#     print(x)

False
False
False
False
False
False
False
False
False
False


In [63]:
limit_repeat

10

In [15]:
games_10000 = analyzeByGAMES(MODEL_10000)

Model: Recon3D.xml
xGAMES time: 23346.919296
xGAMES found an error!


[ErrorSummary(type='type3', errors=[R_ECOAH12: {M_2mp2coa_c} -> {M_3hibutcoa_c}, R_FCLPA: {M_fc1p_c} -> {M_lald__L_c}, R_PYDXNO: {M_pydxn_c} -> {M_pydx_c}, R_TDPGDH: {M_dtdpgal_c=M_dtdpglu_c} -> {M_dtdp4d6dg_c=M_dtdp4d6dm_c}]),
 ErrorSummary(type='echelon', errors=[R_ADCS:  -> {M_4adcho_c}, R_TKT1:  -> {M_s7p_c}, R_HPA3MO: {M_4hphac_c} + {M_etoh_c} -> , R_PALDH: {M_Pald_c} -> , R_DHQTi: {M_3dhq_c} -> , R_DRPA:  -> {M_acald_c}, R_GLUt4: {M_glu__L_e} -> , R_PROt4: {M_pro__L_e} -> , R_THRD_L: {M_thr__L_c} -> , R_HBCHLR:  -> {M_b2coa_c}, R_CHORS: {M_3psme_c} -> , R_ETHAAL: {M_etha_c} -> , R_GMNA: {M_gln__D_c} -> , R_ASNN: {M_asn__L_c} + 0.01 {M_2obut_c=M_2obut_e} + 0.27 {M_2mcit_c=M_micit_c} -> , R_HPOXR: 2.00 {M_h2o2_c} -> , R_PGLYCP: {M_2pglyc_c} -> , R_PENDECACID:  -> 0.77 {M_octedecacid_c}, R_ANS2:  -> {M_Lkynr_c}, R_TETDECACID:  -> 0.97 {M_pendecacid_c}, R_BIWLIP:  -> 0.13 {M_tetdecacid_c}, R_FLAVO2: {M_hh3phac_c} -> , R_ACOADH1: {M_2mbcoa_c} -> , R_DHPS:  -> 0.91 {M_itp_c} + 0.13 {M_

In [21]:
m_1000, s_1000, milp_result_1000, unconserved_metabolites_1000 = analyzeBIGG(MODEL_1000)

Model: iCN718.xml
xGAMES time: 108.660436
xGAMES found an error!


//anaconda3/envs/sbml_env/lib/python3.6/site-packages/scipy/optimize/_linprog_util.py:763: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  warn(redundancy_warning, OptimizeWarning)


LP time: 2.030797
LP found an error!
MILP time: 23.239786


In [61]:
m_4000, s_4000, milp_result_4000, unconserved_metabolites_4000 = analyzeBIGG(MODEL_4000)

Model: iLB1027_lipid.xml
xGAMES time: 1626.331340
xGAMES found an error!
LP time: 272.211592
LP found an error!
MILP time: 285.884376


In [62]:
m_6000, s_6000, milp_result_6000, unconserved_metabolites_6000 = analyzeBIGG(MODEL_6000)

Model: iCHOv1.xml
xGAMES time: 7200.678418
xGAMES found an error!
LP time: 759.267629
LP found an error!
MILP time: 796.065499


In [68]:
len(unconserved_metabolites_6000)

2370

In [71]:
print(s_4000.result.status)
print(s_6000.result.status)

2
2


In [184]:
pulp.cplex_dll_path

'/usr/ilog/cplex/bin/x86_rhel4.0_3.4/libcplex110.so'

In [172]:
import cobra

In [179]:
from SBMLLint.common import util
import libsbml
xml = util.getXML(os.path.join(cn.BIGG_DIR, MODEL_1000))
reader = libsbml.SBMLReader()
document = reader.readSBMLFromString(xml)
model = document.getModel()

In [182]:
cobra.(model,'massBalance', true)

AttributeError: module 'cobra' has no attribute 'verifyModel'

In [104]:
# test roundup
echelon_df = round(m_1000.echelon_df)
rref_df = round(m_1000.getRREFMatrix(echelon_df))

In [105]:
lower_inv = inv(np.round(m.lower))

In [106]:
op_df = m_1000.rref_operation.dot(lower_inv)

In [147]:
op_df.index

Index(['R_G3PD4', 'R_ASPT', 'R_BDH', 'R_PTHPS', 'R_PPPPH', 'R_METTRS',
       'R_IMPC', 'R_EDD', 'R_BACCL2', 'R_CAOPT3',
       ...
       'R_4ABUTD', 'R_ADK1', 'R_CTPS1', 'R_TYRt2r', 'R_GLUabc', 'R_ALDD20y',
       'R_NTPP1', 'R_ALDD2y', 'R_GLNt2r', 'R_LYSabc'],
      dtype='object', length=821)

In [149]:
op_df.columns = op_df.index

In [107]:
m_1000.som_stoichiometry_matrix.shape

(788, 821)

In [151]:
som_reactions = m_1000.convertMatrixToSOMReactions(rref_df)

In [117]:
m.echelon_errors

[]

In [141]:
gr = GAMESReport(m_1000)

In [158]:
reaction_operation = gr.convertOperationSeriesToReactionOperations(op_df.T[res[0]])

In [159]:
reaction_operation[:3]

[ReactionOperation(reaction='R_HMGL_2', operation=-1.0),
 ReactionOperation(reaction='R_BPNT', operation=19.0),
 ReactionOperation(reaction='R_FBP', operation=1.0)]

In [170]:
m_1000.simple.getReaction("R_HMGL_2")

In [169]:
gr.mesgraph.simple.getReaction("R_HMGL_2")

In [171]:
gr.getOperationStoichiometryMatrix(reaction_operation[1:])

AttributeError: 'NoneType' object has no attribute 'reactants'

In [155]:
inferred_reaction = gr.getInferredReaction(reaction_operation)

AttributeError: 'NoneType' object has no attribute 'reactants'

In [129]:
res = []
for reaction in som_reactions:
  if reaction.category == cn.REACTION_ERROR:
    res.append(reaction.label)
print(res)

['R_FBP', 'R_TKT1', 'R_ASPK', 'R_ASPTA', 'R_TALA', 'R_ASAD', 'R_OOAP', 'R_HCO3E', 'R_ASPt2r', 'R_GLCD', 'R_NT5C', 'R_GLUSx_copy2', 'R_DHFR2i', 'R_DHFOR', 'R_MALTt2', 'R_G5SADs', 'R_PSP_L', 'R_FUM']


In [130]:
op_df

,0,1,2,3,4,5,6,7,8,9,...,811,812,813,814,815,816,817,818,819,820
R_G3PD4,2.25000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R_ASPT,0.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R_BDH,-8.25000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R_PTHPS,-1.53125,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
R_PPPPH,3.78125,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R_ALDD20y,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
R_NTPP1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
R_ALDD2y,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
R_GLNt2r,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


'reaction_error'

In [99]:
for col in op_df.columns[:10]:
  result_reaction = op_df

,0,1,2,3,4,5,6,7,8,9,...,811,812,813,814,815,816,817,818,819,820
{M_mqn7_c},-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
{M_fum_c},0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
{M_bhb_c},0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
{M_6pthp_c},0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
{M_gbdp_c},0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
{M_2mcit_c=M_micit_c},5.0,0.0,4.0,1.0,9.0,-7.0,11.0,0.0,8.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
{M_ara5p_c=M_r1p_c=M_r5p_c=M_ru5p__D_c=M_xu5p__D_c=M_xyl__D_c=M_xyl__D_e},-133.0,-1.0,-34.0,43.0,-13.0,-10.0,45.0,0.0,-67.0,-126.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
{M_lac__L_c=M_lac__L_e},0.0,0.0,-1.0,1.0,-1.0,1.0,-2.0,0.0,-2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
{M_2hyoxplac_c=M_34dhpha_c},-1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
(((echelon_df < TOLERANCE) & (echelon_df > 0)).sum()).sum()

6433

In [81]:
m_1000.error_summary

[ErrorSummnary(type='echelon', errors=[R_ASPO1:  -> 0.00 {M_o2_c=M_o2_e} + 0.11 {M_26dap_LL_c=M_26dap__M_c} + 0.14 {M_glc__D_c=M_glc__bD_c} + 0.20 {M_f6p_c=M_g1p_c=M_g6p_B_c=M_g6p_c=M_man1p_c=M_man6p_c} + 0.08 {M_prfp_c=M_prlp_c} + 0.06 {M_asp__D_c=M_asp__D_e=M_asp__L_c} + 0.06 {M_2obut_c=M_2obut_e} + 0.03 {M_ara5p_c=M_r1p_c=M_r5p_c=M_ru5p__D_c=M_xu5p__D_c=M_xyl__D_c=M_xyl__D_e} + 0.06 {M_no2_c=M_no2_e}, R_GLYCLTDy: 0.00 {M_asp__D_c=M_asp__D_e=M_asp__L_c} + 0.00 {M_2obut_c=M_2obut_e} + 0.00 {M_ara5p_c=M_r1p_c=M_r5p_c=M_ru5p__D_c=M_xu5p__D_c=M_xyl__D_c=M_xyl__D_e} + 0.01 {M_no2_c=M_no2_e} -> , R_P5CCD: 0.05 {M_asp__D_c=M_asp__D_e=M_asp__L_c} + 0.12 {M_2obut_c=M_2obut_e} + 0.12 {M_ara5p_c=M_r1p_c=M_r5p_c=M_ru5p__D_c=M_xu5p__D_c=M_xyl__D_c=M_xyl__D_e} + 0.37 {M_no2_c=M_no2_e} -> ])]

In [69]:
# for MODEL_1000
simple.getReaction("R_SEAHCYSHYD_1")

In [15]:
simple.getReaction("R_SEAHCYSHYD")

R_SEAHCYSHYD: M_h2o_c + M_seahcys_c -> M_adn_c + M_selhcys_c

In [16]:
simple.getMolecule("M_h2o_c")

M_h2o_c

In [17]:
m.getNode(simple.getMolecule("M_adn_c"))

{M_adn_c}

In [18]:
error_column = m.lower_inverse.T[m.echelon_errors[0].label]
error_loc = error_column.to_numpy().nonzero()
error_reactions = error_column.index[error_loc]
print(error_reactions)
print(error_column[error_loc[0]])

Index(['R_ENO'], dtype='object')
R_ENO    1.0
Name: R_ENO, dtype: float64


In [19]:
simple.getReaction("R_ENO")

R_ENO: M_2pg_c -> M_h2o_c + M_pep_c

In [22]:
m.getNode(simple.getMolecule("M_2pg_c"))

{M_2pg_c=M_3pg_c}

In [23]:
# select_mat = m.som_stoichiometry_matrix[set(error_reactions)]
# lp_mat = select_mat.loc[(select_mat!=0.0).any(axis=1)].T
# nmet = lp_mat.shape[1]
# # number of reactions
# nreac = lp_mat.shape[0]  
# #  
# b = np.zeros(nreac)
# c = np.ones(nmet)
# # Linear programming. c is constraint (here, zero), 
# # b is vector of possible values for molecule vector. 
# lp_res = linprog(c, A_eq=lp_mat, b_eq=b, bounds=(1, None))
# lp_res